In [1]:
!pip install torch pandas numpy scikit-learn matplotlib seaborn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

    Importing Libraries

In [ ]:
import os, math, random, datetime
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
DATA_CSV = "/kaggle/input/1st-jan-2015-to-30th-sep-2025-district-data/combined_dataset.csv"
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

FEATURE_COLS = ["tp","sro","ssro","swvl1","swvl2","swvl3","t2m","d2m"]
TIME_COL = "time"
DIST_COL = "district"

PAST_SEQ = 168  
HORIZON = 24  
BATCH_SIZE = 32
LR = 1e-3
EPOCHS = 40
K_NEIGHBORS = 4 

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
def ensure_datetime(df, time_col=TIME_COL):
    df[time_col] = pd.to_datetime(df[time_col])
    return df

def date_in_range(ts, start, end):
    return (ts >= pd.to_datetime(start)) & (ts <= pd.to_datetime(end))

In [ ]:
print("Loading CSV:", DATA_CSV)
df = pd.read_csv(DATA_CSV)
df = ensure_datetime(df, TIME_COL)
use_cols = [DIST_COL, TIME_COL] + FEATURE_COLS
df = df[use_cols].drop_duplicates(subset=[DIST_COL, TIME_COL])
print("Loaded rows:", len(df), "unique districts:", df[DIST_COL].nunique())

districts = sorted(df[DIST_COL].unique())
district_to_idx = {d:i for i,d in enumerate(districts)}
num_nodes = len(districts)
print("num_nodes:", num_nodes)

min_time = df[TIME_COL].min().floor('H')
max_time = df[TIME_COL].max().ceil('H')
print("Time range:", min_time, "to", max_time)

full_time_index = pd.date_range(start=min_time, end=max_time, freq='H')

panel = {}
for d in tqdm(districts, desc="Building district panels"):
    sub = df[df[DIST_COL]==d].set_index(TIME_COL).reindex(full_time_index)
    sub_interp = sub[FEATURE_COLS].astype(float).interpolate(method='time', limit_direction='both')
    sub_interp = sub_interp.fillna(method='ffill').fillna(method='bfill').fillna(0.0)
    panel[d] = sub_interp

T = len(full_time_index)
F = len(FEATURE_COLS)
data_array = np.zeros((num_nodes, T, F), dtype=np.float32)
for i,d in enumerate(districts):
    data_array[i] = panel[d].values

print("Data array shape:", data_array.shape)  
train_end = pd.to_datetime("2025-07-31 23:00:00")
val_start = pd.to_datetime("2025-08-01 00:00:00")
val_end = pd.to_datetime("2025-09-30 23:00:00")

train_mask = (full_time_index <= train_end)
val_mask = (full_time_index >= val_start) & (full_time_index <= val_end)
print("Train hours:", train_mask.sum(), "Val hours:", val_mask.sum())

scaler = StandardScaler()
train_data_for_scaler = data_array[:, train_mask, :].reshape(-1, F)
print("Fitting scaler on shape:", train_data_for_scaler.shape)
scaler.fit(train_data_for_scaler)
# apply
data_scaled = np.zeros_like(data_array)
for i in range(num_nodes):
    data_scaled[i] = scaler.transform(data_array[i])

print("Building adjacency matrix via correlation-based kNN (K=", K_NEIGHBORS, ")")
agg_series = data_array.mean(axis=2)
agg_train = agg_series[:, train_mask]

corr = np.corrcoef(agg_train) 
corr = np.nan_to_num(corr)
A = np.zeros((num_nodes, num_nodes), dtype=np.float32)
for i in range(num_nodes):
    neighbors = np.argsort(-corr[i]) 
    cnt = 0
    for j in neighbors:
        if i==j: continue
        A[i,j] = max(0.0, corr[i,j])
        cnt += 1
        if cnt >= K_NEIGHBORS: break
A = (A + A.T) / 2.0
row_sum = A.sum(axis=1, keepdims=True) + 1e-6
A = A / row_sum

print("Adjacency built. Sample row sums (should be ~1):", A.sum(axis=1)[:5])

plt.figure(figsize=(8,6))
sns.heatmap(A, cmap="viridis")
plt.title("Adjacency matrix (correlation kNN)")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "adjacency_heatmap.png"))
plt.close()

class STDataset(Dataset):
    def __init__(self, data_scaled, time_index, mask_hours, past_seq=PAST_SEQ, horizon=HORIZON, nodes_first=True):
        self.data = data_scaled
        self.time_index = time_index
        self.mask = mask_hours
        self.nodes = data_scaled.shape[0]
        self.T = data_scaled.shape[1]
        self.F = data_scaled.shape[2]
        self.past = past_seq
        self.horizon = horizon
        self.indices = []
        for t in range(self.past, self.T - self.horizon + 1):
            input_mask = True
            if not self.mask[t] and not all(self.mask[t-self.past:t]): 
                continue
            if not all(self.mask[t:t+self.horizon]):
                continue
            self.indices.append(t)
        print(f"Built dataset with {len(self.indices)} windows (past={self.past}, horizon={self.horizon}).")
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        t = self.indices[idx]
        x = self.data[:, t-self.past:t, :]   
        y = self.data[:, t:t+self.horizon, :] 
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32), t

train_mask_bool = np.array(train_mask, dtype=bool)
val_mask_bool = np.array(val_mask, dtype=bool)

train_ds = STDataset(data_scaled, full_time_index, train_mask_bool, past_seq=PAST_SEQ, horizon=HORIZON)
val_ds = STDataset(data_scaled, full_time_index, val_mask_bool, past_seq=PAST_SEQ, horizon=HORIZON)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

class TemporalConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size, padding=(kernel_size-1)//2 * dilation, dilation=dilation)
        self.act = nn.ReLU()
        self.bn = nn.BatchNorm1d(out_ch)
    def forward(self, x): 
        return self.bn(self.act(self.conv(x)))

class STGNN(nn.Module):
    def __init__(self, num_nodes, in_feats, hidden_feats=64, horizon=HORIZON, A_init=None):
        super().__init__()
        self.num_nodes = num_nodes
        self.in_feats = in_feats
        self.horizon = horizon

        if A_init is None:
            A_init = np.eye(num_nodes, dtype=np.float32)
        A_tensor = torch.tensor(A_init, dtype=torch.float32)
        self.register_buffer("A_init", A_tensor)
        self.A_weight = nn.Parameter(torch.ones_like(A_tensor) * 0.1) 
        self.input_proj = nn.Linear(in_feats, hidden_feats)
        self.temporal1 = TemporalConvBlock(hidden_feats, hidden_feats, kernel_size=3)
        self.temporal2 = TemporalConvBlock(hidden_feats, hidden_feats, kernel_size=3, dilation=2)
        self.spatial_fc = nn.Linear(hidden_feats, hidden_feats)
        self.forecast = nn.Sequential(
            nn.Linear(hidden_feats, hidden_feats//2),
            nn.ReLU(),
            nn.Linear(hidden_feats//2, horizon * in_feats)
        )

    def forward(self, x):
        B, N, T_in, F = x.shape
        x_proj = self.input_proj(x) 
        hidden = x_proj.permute(0,1,3,2).contiguous()  
        B,N,H,T = hidden.shape
        hidden_reshape = hidden.view(B*N, H, T)
        hidden_t = self.temporal1(hidden_reshape)
        hidden_t = self.temporal2(hidden_t) 
        hidden_t = hidden_t.view(B, N, H, T)
        node_repr = hidden_t.mean(dim=-1)
        A_eff = self.A_init * self.A_weight 
        row_sum = A_eff.sum(dim=1, keepdim=True) + 1e-6
        A_norm = A_eff / row_sum
        agg = torch.einsum("nm, bmh -> bnh", A_norm, node_repr)
        agg = torch.relu(self.spatial_fc(agg))
        out = self.forecast(agg) 
        out = out.view(B, N, self.horizon, F) 
        return out

model = STGNN(num_nodes=num_nodes, in_feats=F, hidden_feats=128, horizon=HORIZON, A_init=A).to(DEVICE)
print(model)
print("Number of parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
criterion = nn.MSELoss()

train_losses = []
val_losses = []
best_val = 1e18
save_model_path = os.path.join(OUTPUT_DIR, "stgnn_best.pt")

for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} (train)", leave=False)
    for xb, yb, t_idxs in pbar:
        xb = xb.to(DEVICE)   
        yb = yb.to(DEVICE)   
        optimizer.zero_grad()
        yhat = model(xb)    
        loss = criterion(yhat, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
        pbar.set_postfix({"loss": loss.item()})
    epoch_train_loss = running_loss / len(train_ds)
    train_losses.append(epoch_train_loss)

    model.eval()
    val_running = 0.0
    all_y = []
    all_yhat = []
    with torch.no_grad():
        for xb, yb, t_idxs in val_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            yhat = model(xb)
            loss = criterion(yhat, yb)
            val_running += loss.item() * xb.size(0)
            all_y.append(yb.cpu().numpy())
            all_yhat.append(yhat.cpu().numpy())
    epoch_val_loss = val_running / len(val_ds) if len(val_ds)>0 else np.nan
    val_losses.append(epoch_val_loss)
    print(f"Epoch {epoch} train_loss={epoch_train_loss:.6f} val_loss={epoch_val_loss:.6f}")

    if epoch_val_loss < best_val:
        best_val = epoch_val_loss
        torch.save({"model_state": model.state_dict(),
                    "scaler": scaler, "config": {"past":PAST_SEQ, "horizon":HORIZON, "features": FEATURE_COLS, "districts":districts}},
                   save_model_path)
        print("Saved best model to", save_model_path)

plt.figure()
plt.plot(train_losses, label="train_loss")
plt.plot(val_losses, label="val_loss")
plt.xlabel("epoch"); plt.ylabel("MSE loss"); plt.legend(); plt.title("Training curves")
plt.savefig(os.path.join(OUTPUT_DIR, "training_curves.png"))
plt.close()

print("Running detailed validation visualization (using best model)")
ckpt = torch.load(save_model_path, map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
model.eval()

N_plot_batches = 6
val_iter = iter(val_loader)
sample_count = 0
all_preds = []
all_trues = []
sample_meta = []
with torch.no_grad():
    for bidx in range(N_plot_batches):
        try:
            xb, yb, t_idxs = next(val_iter)
        except StopIteration:
            break
        xb = xb.to(DEVICE); yb = yb.to(DEVICE)
        yhat = model(xb)
        all_preds.append(yhat.cpu().numpy()) 
        all_trues.append(yb.cpu().numpy())
        sample_meta.append(t_idxs.numpy())
        sample_count += xb.size(0)

if len(all_preds)==0:
    print("No validation windows found for plotting.")
else:
    preds = np.concatenate(all_preds, axis=0)[:100] 
    trues = np.concatenate(all_trues, axis=0)[:100]
    S = preds.shape[0]
    preds_reshaped = preds.reshape(-1, F)
    trues_reshaped = trues.reshape(-1, F)
    preds_inv = scaler.inverse_transform(preds_reshaped).reshape(S, num_nodes, HORIZON, F)
    trues_inv = scaler.inverse_transform(trues_reshaped).reshape(S, num_nodes, HORIZON, F)
    metrics = {}
    for fi, fname in enumerate(FEATURE_COLS):
        y_true_f = trues_inv[:,:,:,fi].ravel()
        y_pred_f = preds_inv[:,:,:,fi].ravel()
        mse = mean_squared_error(y_true_f, y_pred_f)
        metrics[fname] = {"mse": mse}
    print("Validation metrics (sampled windows):")
    for k,v in metrics.items():
        print(f"  {k}: MSE={v['mse']:.4f}")
    plot_district_indices = [0, max(0,num_nodes//3), max(0, 2*num_nodes//3), num_nodes-1]
    for d_idx in plot_district_indices:
        for fi, fname in enumerate(FEATURE_COLS):
            plt.figure(figsize=(10,4))
            s = 0
            t = 0
            true_ts = trues_inv[s, d_idx, :, fi]
            pred_ts = preds_inv[s, d_idx, :, fi]
            hours = np.arange(HORIZON)
            plt.plot(hours, true_ts, label="true", marker='o')
            plt.plot(hours, pred_ts, label="pred", marker='x')
            plt.title(f"District={districts[d_idx]} | feature={fname} | horizon={HORIZON}h")
            plt.xlabel("hours ahead"); plt.ylabel(fname)
            plt.legend()
            outfn = os.path.join(OUTPUT_DIR, f"ts_d{d_idx}_{districts[d_idx]}_{fname}.png")
            plt.savefig(outfn); plt.close()
    for fi, fname in enumerate(FEATURE_COLS):
        y_true_f = trues_inv[:,:,:,fi].ravel()
        y_pred_f = preds_inv[:,:,:,fi].ravel()
        plt.figure(figsize=(6,6))
        sns.scatterplot(x=y_true_f, y=y_pred_f, s=10, alpha=0.3)
        plt.plot([y_true_f.min(), y_true_f.max()], [y_true_f.min(), y_true_f.max()], 'r--')
        plt.xlabel("true"); plt.ylabel("pred")
        plt.title(f"Scatter: {fname}")
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"scatter_{fname}.png"))
        plt.close()
    node_errors = []
    for n in range(num_nodes):
        y_t = trues_inv[:,:, :, :].reshape(-1, F)[:, :]
    per_node_rmse = []
    for n in range(num_nodes):
        y_true_n = trues_inv[:, n, :, :].reshape(-1, F)
        y_pred_n = preds_inv[:, n, :, :].reshape(-1, F)
        rmse = np.sqrt(((y_true_n - y_pred_n)**2).mean())
        per_node_rmse.append(rmse)
    plt.figure(figsize=(8,3))
    sns.barplot(x=districts, y=per_node_rmse)
    plt.xticks(rotation=90)
    plt.title("Per-district RMSE (sampled)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "per_district_rmse.png"))
    plt.close()
    
    for fi,fname in enumerate(FEATURE_COLS):
        res = (trues_inv[:,:,:,fi] - preds_inv[:,:,:,fi]).ravel()
        plt.figure(figsize=(6,4))
        sns.histplot(res, bins=80, kde=True)
        plt.title(f"Residuals histogram: {fname}")
        plt.savefig(os.path.join(OUTPUT_DIR, f"resid_hist_{fname}.png"))
        plt.close()

print("All plots saved to", OUTPUT_DIR)
print("Script finished.")

Using device: cuda
Loading CSV: /kaggle/input/1st-jan-2015-to-30th-sep-2025-district-data/combined_dataset.csv


/tmp/ipykernel_18/3502595890.py:79: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  min_time = df[TIME_COL].min().floor('H')
/tmp/ipykernel_18/3502595890.py:80: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  max_time = df[TIME_COL].max().ceil('H')
/tmp/ipykernel_18/3502595890.py:83: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_time_index = pd.date_range(start=min_time, end=max_time, freq='H')


Loaded rows: 1121748 unique districts: 12
num_nodes: 12
Time range: 2015-01-01 00:00:00 to 2025-09-30 23:00:00


Building district panels:   0%|          | 0/12 [00:00<?, ?it/s]/tmp/ipykernel_18/3502595890.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_interp = sub_interp.fillna(method='ffill').fillna(method='bfill').fillna(0.0)
Building district panels:   8%|▊         | 1/12 [00:00<00:01,  6.38it/s]/tmp/ipykernel_18/3502595890.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_interp = sub_interp.fillna(method='ffill').fillna(method='bfill').fillna(0.0)
Building district panels:  17%|█▋        | 2/12 [00:00<00:01,  6.70it/s]/tmp/ipykernel_18/3502595890.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub_interp = sub_interp.fillna(method='ffill').fillna(method='bfill').fillna(0.0)
Building district panels

Data array shape: (12, 94224, 8)
Train hours: 92760 Val hours: 1464
Fitting scaler on shape: (1113120, 8)
Building adjacency matrix via correlation-based kNN (K= 4 )
Adjacency built. Sample row sums (should be ~1): [0.9999999  0.99999976 0.9999998  0.9999998  0.9999995 ]
Built dataset with 92569 windows (past=168, horizon=24).
Built dataset with 1441 windows (past=168, horizon=24).
STGNN(
  (input_proj): Linear(in_features=8, out_features=128, bias=True)
  (temporal1): TemporalConvBlock(
    (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (act): ReLU()
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (temporal2): TemporalConvBlock(
    (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (act): ReLU()
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (spatial_fc): Linear(in_features=128, out_features=128, bias=True)
  (forecast): S

...

Epoch 37 train_loss=0.068726 val_loss=0.391472


Epoch 38 train_loss=0.068942 val_loss=0.392157


Epoch 39 train_loss=0.068487 val_loss=0.380365


Epoch 40 train_loss=0.067633 val_loss=0.408765
Running detailed validation visualization (using best model)


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL sklearn.preprocessing._data.StandardScaler was not an allowed global by default. Please use `torch.serialization.add_safe_globals([StandardScaler])` or the `torch.serialization.safe_globals([StandardScaler])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types 